# Import libraries

In [1]:
# for requesting data from Yahoo Finance API and NASDAQ API
import json
import ast
import requests
from requests.exceptions import Timeout
from config import x_rapidapi_key, x_rapidapi_host

# for data extraction and analyses
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime

# for data visualisation
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

# Collecting financial data for stocks

In [2]:
# data source URL for list of ticker symbols from NASDAQ:
# "https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true"
# data copied and pasted from URL to tickers.txt (dictionary string) but will write code to automate pulling data from URL
tickers = open("tickers.txt", "r")
contents = tickers.read()
contents = contents[:2429232]

# convert dictionary string to dictionary
ticker_dict = json.loads(contents)
ticker_dict

{'data': {'headers': {'symbol': 'Symbol',
   'name': 'Name',
   'lastsale': 'Last Sale',
   'netchange': 'Net Change',
   'pctchange': '% Change',
   'marketCap': 'Market Cap',
   'country': 'Country',
   'ipoyear': 'IPO Year',
   'volume': 'Volume',
   'sector': 'Sector',
   'industry': 'Industry',
   'url': 'Url'},
  'rows': [{'symbol': 'A',
    'name': 'Agilent Technologies Inc. Common Stock',
    'lastsale': '$149.27',
    'netchange': '-0.24',
    'pctchange': '-0.161%',
    'volume': '1456502',
    'marketCap': '45294970196.00',
    'country': 'United States',
    'ipoyear': '1999',
    'industry': 'Electrical Products',
    'sector': 'Capital Goods',
    'url': '/market-activity/stocks/a'},
   {'symbol': 'AA',
    'name': 'Alcoa Corporation Common Stock ',
    'lastsale': '$35.62',
    'netchange': '-1.67',
    'pctchange': '-4.478%',
    'volume': '8066765',
    'marketCap': '6651135417.00',
    'country': '',
    'ipoyear': '2016',
    'industry': 'Metal Fabrications',
    'se

In [3]:
# extract list of unique countries from json file ticker_dict
countries = list(set([x["country"] for x in ticker_dict["data"]["rows"]]))

# extract symbols from the json file ticker_dict for companies listed in the USA
tickers = [x["symbol"] for x in ticker_dict["data"]["rows"] if x["country"] == "United States"]

# extract company names from ticker_dict for companies listed in the USA
names = [x["name"] for x in ticker_dict["data"]["rows"] if x["country"] == "United States"]

In [4]:
# data source URLs (company financial data and stock stats), using Yahoo Finance
url_financials = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-financials"
url_stats = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-statistics"

# define the region
region = "US"

In [5]:
def ticker_info(url, symbol):
    querystring = {"symbol": symbol, 
                   "region": region}
    headers = {"x-rapidapi-key": x_rapidapi_key,
               "x-rapidapi-host": x_rapidapi_host}
    try:
        response = requests.get(url = url, 
                                headers = headers, 
                                params = querystring,
                                timeout = 5)
    except Timeout:    
        print("The request has timed out")
    else:
        return response.json()

In [6]:
# get financial information about AAPL (Apple Inc) using the ticker_info function
# to find out what information to use
financials = ticker_info(url_financials, "AAPL")

In [7]:
# determine the keys in the json output (dictionary)
financials.keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews', 'timeSeries', 'meta', 'loading', 'errorList'])

In [8]:
def cashflow_df(ticker):
    """ Generates a dataframe containing information from cash flow statements """ 
    # Get financial information using the ticker_info function from Yahoo Finance
    financials = ticker_info(url_financials, ticker)
    
    # Extract selected keys and values and put them in lists
    cashflow = []
    for x in financials["cashflowStatementHistory"]["cashflowStatements"]:
        keys = []
        values = []
        for key, value in x.items():
            keys.append(key)
            if isinstance(value, dict):
                if key == "endDate":
                    values.append(value.get("fmt"))
                else:    
                    values.append(value.get("raw"))
            else:
                values.append(value)

        # Create dictionary containing keys and values   
        dict_ = {key: value for key, value in zip(keys, values)}
        cashflow.append(dict_)

    # Convert the dictionary into a dataframe
    df = pd.DataFrame(cashflow)
    df["endDate"] = [x[0:4] for x in df["endDate"]]
    df["ticker"] = ticker
    return df

In [9]:
def earnings_df(ticker):
    """ Generates a dataframe containing information from earnings statements """
    # Get financial information using the ticker_info function from Yahoo Finance
    financials = ticker_info(url_financials, ticker)
    
    # Extract annual earnings information from Yahoo Finance
    earnings = []
    for x in financials["earnings"]["financialsChart"]["yearly"]:
        keys = []
        values = []
        for key, value in x.items():
            keys.append(key)
            if isinstance(value, dict):
                values.append(value.get("raw"))
            else:
                values.append(value)

        # Convert earnings information into dictionary
        dict_ = {key:value for key,value in zip(keys,values)}
        earnings.append(dict_)

    # Convert list of dictionaries to pandas dataframe
    df = pd.DataFrame(earnings)
    df["ticker"] = ticker
    return df

In [10]:
def income_df(ticker):
    """ Generates a dataframe containing information from income statements """
    # Get financial information using the ticker_info function from Yahoo Finance
    financials = ticker_info(url_financials, ticker)
    
    # Extract annual income statement data from Yahoo Finance
    incomes = []
    for x in financials["incomeStatementHistory"]["incomeStatementHistory"]:
        keys = []
        values = []
        for key, value in x.items():
            keys.append(key)
            if isinstance(value, dict):
                if key == "endDate":
                    values.append(value.get("fmt"))
                else:   
                    values.append(value.get("raw"))
            else:
                values.append(value)

         # Convert income statement information into dictionary
        dict_ = {key:value for key,value in zip(keys,values)}
        incomes.append(dict_)

    # Convert list of dictionaries to pandas dataframe
    df = pd.DataFrame(incomes)
    df["endDate"] = [x[0:4] for x in df["endDate"]]
    df["ticker"] = ticker
    return df   

In [11]:
def balance_df(ticker):
    """ Generates a dataframe containing information from income statements """
    # Get financial information using the ticker_info function from Yahoo Finance
    financials = ticker_info(url_financials, ticker)
    
    # Extract annual balance statement data from Yahoo Finance
    balances = []
    for x in financials["balanceSheetHistory"]["balanceSheetStatements"]:
        keys = []
        values = []
        for key, value in x.items():
            keys.append(key)
            if isinstance(value, dict):
                if key == "endDate":
                    values.append(value.get("fmt"))
                else:    
                    values.append(value.get("raw"))
            else:
                values.append(value)

        # Convert balance sheet information into dictionary
        dict_ = {key:value for key,value in zip(keys,values)}
        balances.append(dict_)

    # Convert list of dictionaries into pandas dataframe
    df = pd.DataFrame(balances)
    df["endDate"] = [x[0:4] for x in df["endDate"]]
    df["ticker"] = ticker
    return df

In [12]:
cashflow_df("AVGO").columns

Index(['changeToLiabilities', 'totalCashflowsFromInvestingActivities',
       'netBorrowings', 'totalCashFromFinancingActivities',
       'changeToOperatingActivities', 'issuanceOfStock', 'netIncome',
       'changeInCash', 'endDate', 'repurchaseOfStock',
       'totalCashFromOperatingActivities', 'depreciation',
       'otherCashflowsFromInvestingActivities', 'dividendsPaid',
       'changeToInventory', 'changeToAccountReceivables',
       'otherCashflowsFromFinancingActivities', 'maxAge', 'changeToNetincome',
       'capitalExpenditures', 'investments', 'ticker'],
      dtype='object')

In [13]:
earnings_df = earnings_df("AVGO")
earnings_df

,date,revenue,earnings,ticker
0,2017,17636000000,1692000000,AVGO
1,2018,20848000000,12259000000,AVGO
2,2019,22597000000,2724000000,AVGO
3,2020,23888000000,2960000000,AVGO


In [14]:
income_df("AVGO").columns

Index(['researchDevelopment', 'effectOfAccountingCharges', 'incomeBeforeTax',
       'minorityInterest', 'netIncome', 'sellingGeneralAdministrative',
       'grossProfit', 'ebit', 'endDate', 'operatingIncome',
       'otherOperatingExpenses', 'interestExpense', 'extraordinaryItems',
       'nonRecurring', 'otherItems', 'incomeTaxExpense', 'totalRevenue',
       'totalOperatingExpenses', 'costOfRevenue', 'totalOtherIncomeExpenseNet',
       'maxAge', 'discontinuedOperations', 'netIncomeFromContinuingOps',
       'netIncomeApplicableToCommonShares', 'ticker'],
      dtype='object')

In [15]:
# Create a dataframe containing income of a specific ticker
income_df = income_df("AVGO")

# Create a list of cost of goods sold (formula: COGS = gross profit - revenue)
cogs = list(income_df["grossProfit"] - income_df["totalRevenue"])

# Calculate proportion of COGS in terms of total revenue
cogs_pct = [(cogs[x] / income_df["totalRevenue"][x]) * 100 
            for x in range(0,len(income_df["totalRevenue"]))]

# Calculate proportion of operating expenses to total revenue
opex_pct = list((income_df["totalOperatingExpenses"]/ income_df["totalRevenue"]) * 100)

In [29]:
# Calculate 

[82.20864032150034, 80.83373899190158, 73.68092862624712, 84.75277840780223]

In [ ]:
income_df["grossProfit"][0] - income_df["totalRevenue"][0]

In [30]:
balance_df("AVGO").columns

Index(['intangibleAssets', 'capitalSurplus', 'totalLiab',
       'totalStockholderEquity', 'otherCurrentLiab', 'totalAssets', 'endDate',
       'otherCurrentAssets', 'otherLiab', 'goodWill', 'treasuryStock',
       'otherAssets', 'cash', 'totalCurrentLiabilities',
       'deferredLongTermAssetCharges', 'shortLongTermDebt',
       'otherStockholderEquity', 'propertyPlantEquipment',
       'totalCurrentAssets', 'netTangibleAssets', 'netReceivables', 'maxAge',
       'longTermDebt', 'inventory', 'accountsPayable', 'commonStock',
       'retainedEarnings', 'minorityInterest', 'ticker'],
      dtype='object')

In [ ]:
# Create a list of revenues from the earnings df
actual_revenues = list(earnings_df["revenue"])
actual_years = list(earnings_df["date"])

# Calculate year-on-year revenues growth
yoy_growth = [round((((actual_revenues[i+1] / actual_revenues[i]) - 1) * 100),2) for i in range(0,len(actual_revenues)-1)]
yoy_growth.insert(0, "NA") # first year does not have yoy growth stat

In [ ]:
# Create a list of forecast years (duration: 5 years)
forecast_years = []
for i in range(1, 6):
    fyr = actual_years[-1] + i
    i += 1
    forecast_years.append(fyr)

In [ ]:
# Based on the range of yoy growth, setting the following as forecasted growth rates:
optimistic_case = round(max(yoy_growth[1:]))
base_case = round(sum(yoy_growth[1:])/len(yoy_growth[1:]))
worst_case = round(min(yoy_growth[1:]))

In [ ]:
# Create a function that will calculate revenue growth forecasts 
# based on any of the three growth rates
def forecast_revgrowth(case, revenue):
    growth = round(revenue * ((case / 100) + 1))
    return growth

In [ ]:
def forecast_revgrowth2(case):
    forecast_revenues = []
    for i in range(-1, 4):
        if i < 0:
            growth = forecast_revgrowth(case, actual_revenues[i])
            forecast_revenues.append(growth)
        else:
            growth = forecast_revgrowth(case, forecast_revenues[i])
            forecast_revenues.append(growth)

    return forecast_revenues    

In [ ]:
forecast_revgrowth2(base_case)